In [1]:
from picsellia import Client
import main
from util.infer import infer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

Client initialization

In [3]:
clt = Client(token="463389a8-52bd-4fd3-bc0a-9198d43fe76b")

Initializing Picsell.ia Client
Connection Established
It's your first training for this project
Creating directory for project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/
Creating directory for PNG Images of project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/
Creating directory for checkpoints project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/
Creating directory for logs of project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/logs/
Creating directory for records of project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/
Creating directory for config of project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/
Creating directory for results of project 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/results/


In [4]:
model_selected = "models/mask_rcnn/"
ckpt_dir = clt.checkpoint_dir
conf_dir = clt.config_dir

In [5]:
clt.dl_annotations()
clt.generate_labelmap()
clt.local_pic_save(prop=0.5) 
main.create_record_files(label_path=clt.label_path, 
                         record_dir=clt.record_dir,  
                         tfExample_generator=clt.tf_vars_generator)

Annotations pulled ...
Generating labelmap ...
Label_map.pbtxt crée @ 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0//label_map.pbtxt
['label1', 'label2']
[[1 1 1 1 1 0]
 [0 0 0 0 1 1]]
[0. 1. 1. 1. 0. 1.]
4 Images used for training, 2 Images used for validation
0 files were already on your machine
 6 PNG images have been downloaded to your machine
Sending repartition to Picsell.ia backend
Repartition send ..


ok
ok
ok
ok
ok
ok
ok
ok
Successfully created the TFRecords: 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/records/train.record
ok
ok
ok
ok
Successfully created the TFRecords: 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/records/eval.record


In [6]:
main.edit_config(model_selected=model_selected, 
                 config_output_dir=conf_dir, 
                 record_dir=clt.record_dir, 
                 label_map_path=clt.label_path, 
                 masks="PNG_MASKS", num_steps=20)



INFO:tensorflow:Writing pipeline config file to 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/config/pipeline.config


In [7]:
main.legacy_train(ckpt_dir=ckpt_dir, conf_dir=conf_dir)

m] is not available in checkpoint
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/mask_rcnn/model.ckpt
INFO:tensorflow:Restoring parameters from models/mask_rcnn/model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/checkpoint/model.ckpt
INFO:tensorflow:Saving checkpoint to path 463389a8-52bd-4fd3-bc0a-9198d43fe76b/0/checkpoint/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:Startin

In [9]:
dict_log = main.tfevents_to_dict(path=ckpt_dir)
# clt.send_logs(dict_log)

In [10]:
main.export_infer_graph(ckpt_dir=ckpt_dir, 
                        exported_model_dir=clt.exported_model, 
                        pipeline_config_path=conf_dir,
                        write_inference_graph=False, input_type="image_tensor", input_shape=None)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims` instead.
Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims` instead.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Removing feature_maps from prediction_dict
INFO:tensorflow:Removing feature_maps from prediction_dict
Instructions for updating:
Use `tf.profile

In [11]:
exported_model_path=clt.exported_model
infer(clt.eval_list, exported_model_path=exported_model_path, 
        label_map_path=clt.label_path, 
        results_dir=clt.results_dir ,
        min_score_thresh=0.2)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.


In [50]:
# train_id = clt.training_id 
# main.edit_config(model_selected=model_selected, 
#                  config_output_dir=conf_dir, 
#                  record_dir=clt.record_dir, 
#                  label_map_path=clt.label_path, 
                #  masks="PNG_MASKS", num_steps=20, training_id=train_id)